# Collisions Study - Step-by-Step

This notebook walks through analyzing motor vehicle collisions in Downtown Brooklyn using UrbanMapper.
We’ll load collision data, map it to intersections, count collisions per node, and visualise the results.

## Data Sources

- **[NYC DOT Motor Vehicle Collisions](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)**  


⚠️ Please Note — Within The Documentation's Interactive Examples ⚠️

First and foremost, please bear with us; some of our Jupyter Notebooks cannot be interactive and are thus displayed as is in the documentation.  Feel free to install the library and test it out locally.  Next, determine whether they are interactive, which means you can see the output of each cell.  As a result, because it is not a good practice to save datasets in a GitHub (or any other Git in general) repository, we attempted to import urban datasets from `HuggingFace` using `from_huggingface(.)` rather than `from_file(.)`, which would need local file availability.  Nonetheless, this was (1) not always viable (certain datasets are not on `HuggingFace`), and (2) this does not preclude you from using `from_file(.)` or any other available via the API reference's `Loader` module.

In [ ]:
import urban_mapper as um

# Initialise UrbanMapper
mapper = um.UrbanMapper()

# Step 1: Create urban layer for intersections
layer = (
    mapper.urban_layer
    .with_type("streets_intersections")
    .from_place("Downtown Brooklyn, New York City, USA", network_type="drive")
    .build()
)

In [ ]:
# Step 2: Load collision data
# Note: For the documentation interactive mode, we only query 5000 records from the dataset.  Feel free to remove for a more realistic analysis.
data = (
    mapper.loader
    .from_huggingface("oscur/NYC_vehicle_collisions", number_of_rows=5000, streaming=True)
    .with_columns(longitude_column="LONGITUDE", latitude_column="LATITUDE")
    .load()
)

data['LONGITUDE'] = data['LONGITUDE'].astype(float)
data['LATITUDE'] = data['LATITUDE'].astype(float)

In [ ]:
# Step 3: Impute missing coordinates
imputer = (
    mapper.imputer
    .with_type("SimpleGeoImputer")
    .on_columns(longitude_column="LONGITUDE", latitude_column="LATITUDE",)
    .build()
)
data = imputer.transform(data, layer)

In [ ]:
# Step 4: Filter to bounding box
filter_step = mapper.filter.with_type("BoundingBoxFilter").build()
data = filter_step.transform(data, layer)

In [ ]:
# Step 5: Map to nearest intersections
_, mapped_data = layer.map_nearest_layer(
    data,
    longitude_column="LONGITUDE",
    latitude_column="LATITUDE",
    output_column="nearest_intersection"
)

In [ ]:

# Step 6: Enrich with collision counts
enricher = (
    mapper.enricher
    .with_data(group_by="nearest_intersection")
    .count_by(output_column="collision_count")
    .build()
)
enriched_layer = enricher.enrich(mapped_data, layer)

In [ ]:
# Step 7: Visualize interactively
visualiser = (
    mapper.visual
    .with_type("Interactive")
    .with_style({"tiles": "CartoDB dark_matter", "colorbar_text_color": "white"})
    .build()
)
fig_interactive = visualiser.render(enriched_layer.get_layer(), columns=["collision_count"])
fig_interactive

In [ ]:
# Step 8: Visualize statically
visualiser = mapper.visual.with_type("Static").build()
fig_static = visualiser.render(enriched_layer.get_layer(), columns=["collision_count"])